In [46]:
#imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import leaguedashplayerbiostats
from nba_api.stats.endpoints import playerdashboardbygeneralsplits
from nba_api.stats.endpoints import playervsplayer

In [ ]:
def player_id(player_name):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_id = players_data[players_data['PLAYER_NAME'] == player_name].get('PLAYER_ID')
    return player_id

In [38]:
#function to get player gamelog based on Player Name rather than ID

def player_gamelog(player_name):
    id = player_id(player_name)
    gamelog = playergamelog.PlayerGameLog(player_id= id)
    return gamelog.get_data_frames()[0]

player_gamelog('Nikola Jokic')

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22023,203999,0022301193,"APR 14, 2024",DEN @ MEM,W,31,7,11,0.636,...,11,15,5,4,0,4,1,15,19,1
1,22023,203999,0022301180,"APR 12, 2024",DEN @ SAS,L,39,9,13,0.692,...,5,7,12,2,1,3,5,22,-1,1
2,22023,203999,0022301164,"APR 10, 2024",DEN vs. MIN,W,38,16,20,0.800,...,9,11,7,3,0,2,5,41,6,1
3,22023,203999,0022301154,"APR 09, 2024",DEN @ UTA,W,34,12,20,0.600,...,8,13,7,3,1,5,2,28,23,1
4,22023,203999,0022301130,"APR 06, 2024",DEN vs. ATL,W,31,6,13,0.462,...,13,14,11,2,0,4,1,19,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,22023,203999,0022300123,"NOV 01, 2023",DEN @ MIN,L,30,11,23,0.478,...,8,10,3,0,0,5,0,25,-19,1
75,22023,203999,0022300110,"OCT 30, 2023",DEN vs. UTA,W,35,12,16,0.750,...,9,10,11,0,2,1,3,27,13,1
76,22023,203999,0022300095,"OCT 29, 2023",DEN @ OKC,W,30,12,16,0.750,...,12,14,5,0,0,4,3,28,21,1
77,22023,203999,0022300078,"OCT 27, 2023",DEN @ MEM,W,35,8,14,0.571,...,9,12,7,1,1,9,1,22,6,1


In [ ]:
def playerVplayer(player_name, vs_player_name):
    id = player_id(player_name)
    vid = player_id(vs_player_name)
    gamelog = playervsplayer.PlayerVsPlayer(vs_player_id=vid, player_id=id)
    return gamelog.get_data_frames()

playerVplayer('Nikola Jokic', 'Rudy Gobert')

In [55]:
# Testing accuracy for different ML models on point predictions
kingStats = player_gamelog('LeBron James')
# target
y = kingStats.PTS
# Features should be considered carefully, I just threw in ones I think make sense
# Obviously some of these like REB, AST, STL, etc probably don't make sense since we can't know these things before the game, although we might know what his average stats are for those categories before the game
kingStats_features = ['MIN', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'FGA', 'FGM']
X = kingStats[kingStats_features]
# Splitting data to test for accuracy
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

# DecisionTreeRegressor test
kingStats_model = DecisionTreeRegressor()
kingStats_model.fit(train_X, train_y)
val_predictions = kingStats_model.predict(val_X)
# ~2.833 MAE, <- haven't really put anything in so would not trust that
print(mean_absolute_error(val_y, val_predictions))


2.8333333333333335
